In [11]:
import os
import zipfile
import shutil
import io
import json
import re


In [ ]:
json

In [83]:
from TextSAM import TextSAM

In [113]:
import TextSAM

In [109]:
dlpk_path = r"C:\Users\rami8629\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\GeoAi & Deep Learning\DLPKs\TextSAM.dlpk"
zip_path = dlpk_path.replace('.dlpk', '.zip')

try:
    # Rename the .dlpk file to .zip
    shutil.move(dlpk_path, zip_path)

    # Extract and print .py file names from the main directory of the .zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_names = zip_ref.namelist()
        emd_files = [file_name for file_name in file_names if file_name.endswith('.emd') and '/' not in file_name]
        py_files = [file_name for file_name in file_names if file_name.endswith('.py') and '/' not in file_name]
        
        if emd_files:
            emd_file = emd_files[0]
            class_name = os.path.splitext(emd_file)[0]  # Extract class name from .emd file name
            with zip_ref.open(emd_file) as file:
                emd_content = file.read().decode('utf-8')
                emd_data = json.loads(emd_content)
                image_height = emd_data.get("ImageHeight")
                if image_height is not None:
                    image_height_value = int(image_height)
                    padding = image_height_value // 4
                    print(f"Extracted ImageHeight value: {padding}")

        if py_files:
            py_file = py_files[0]
            with zip_ref.open(py_file) as file:
                py_content = file.read().decode('utf-8')
                # Use regex to find the dynamically named class and getParameterInfo function
                class_pattern = rf'class\s+{class_name}\s*:\s*(.*?)(?=class\s|\Z)'
                class_match = re.search(class_pattern, py_content, re.DOTALL)
                if class_match:
                    class_content = class_match.group(1)
                    function_match = re.search(r'def\s+getParameterInfo\s*\(.*?\)\s*:\s*(.*?)(?=def\s|\Z)', class_content, re.DOTALL)
                    if function_match:
                        function_content = function_match.group(0)
                        print("Extracted getParameterInfo function:")
                        print(function_content)
                    else:
                        print(f"getParameterInfo function not found in {class_name} class.")
                else:
                    print(f"{class_name} class not found in the .py file.")
                
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Rename the .zip file back to .dlpk
    if os.path.exists(zip_path):
        shutil.move(zip_path, dlpk_path)

Extracted ImageHeight value: 256
Extracted getParameterInfo function:
def getParameterInfo(self):
        required_parameters = [
            {
                "name": "raster",
                "dataType": "raster",
                "required": True,
                "displayName": "Raster",
                "description": "Input Raster",
            },
            {
                "name": "model",
                "dataType": "string",
                "required": True,
                "displayName": "Input Model Definition (EMD) File",
                "description": "Input model definition (EMD) JSON file",
            },
            {
                "name": "device",
                "dataType": "numeric",
                "required": False,
                "displayName": "Device ID",
                "description": "Device ID",
            },
        ]
        required_parameters.extend(
            [
                {
                    "name": "text_prompt",
                    "dataT

In [55]:
required_parameters_content

'[\r\n                {\r\n                    "name": "text_prompt",\r\n                    "dataType": "string",\r\n                    "required": False,\r\n                    "value": "",\r\n                    "displayName": "Text Prompt",\r\n                    "description": "Text Prompt",\r\n                },\r\n                {\r\n                    "name": "padding",\r\n                    "dataType": "numeric",\r\n                    "value": int(self.json_info["ImageHeight"]) // 4,\r\n                    "required": False,\r\n                    "displayName": "Padding",\r\n                    "description": "Padding",\r\n                },\r\n                {\r\n                    "name": "batch_size",\r\n                    "dataType": "numeric",\r\n                    "required": False,\r\n                    "value": 4,\r\n                    "displayName": "Batch Size",\r\n                    "description": "Batch Size",\r\n                },\r\n                {